In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.pyplot as plt


%matplotlib inline


In [29]:
df = pd.read_csv('D:\jupyter\projects\data\healthcare-dataset-stroke-data.csv')

In [30]:
df.head()

gender   age  hypertension  heart_disease ever_married      work_type  \
0    Male  67.0             0              1          Yes        Private   
1  Female  61.0             0              0          Yes  Self-employed   
2    Male  80.0             0              1          Yes        Private   
3  Female  49.0             0              0          Yes        Private   
4  Female  79.0             1              0          Yes  Self-employed   

  Residence_type  avg_glucose_level   bmi   smoking_status  stroke  
0          Urban             228.69  36.6  formerly smoked       1  
1          Rural             202.21   NaN     never smoked       1  
2          Rural             105.92  32.5     never smoked       1  
3          Urban             171.23  34.4           smokes       1  
4          Rural             174.12  24.0     never smoked       1

---

#### Describe the numerical data ?

### Q: How many missing values are there in each column ?

In [43]:
print(df.isnull().sum())

gender                 0
age                    0
hypertension           0
heart_disease          0
ever_married           0
work_type              0
Residence_type         0
avg_glucose_level      0
bmi                  201
smoking_status         0
stroke                 0
dtype: int64


In [45]:
median_bmi = df["bmi"].median()
df['bmi'].fillna(median_bmi, inplace=True)
print(df.isnull().sum())

gender               0
age                  0
hypertension         0
heart_disease        0
ever_married         0
work_type            0
Residence_type       0
avg_glucose_level    0
bmi                  0
smoking_status       0
stroke               0
dtype: int64


In [54]:
df['avg_glucose_level'].round(0)
df['age'].round(0)
df['bmi'].round(0)

0       37.0
1       28.0
2       32.0
3       34.0
4       24.0
        ... 
5104    28.0
5105    40.0
5106    31.0
5107    26.0
5108    26.0
Name: bmi, Length: 5109, dtype: float64

In [58]:
df.dtypes


gender                object
age                  float64
hypertension           int64
heart_disease          int64
ever_married          object
work_type             object
Residence_type        object
avg_glucose_level    float64
bmi                  float64
smoking_status        object
stroke                 int64
dtype: object

In [68]:
df.avg_glucose_level = df.avg_glucose_level.astype(int)
df.age = df.age.astype(int)
df.bmi = df.bmi.astype(int)



In [69]:
df.dtypes


gender               object
age                   int32
hypertension          int64
heart_disease         int64
ever_married         object
work_type            object
Residence_type       object
avg_glucose_level     int32
bmi                   int32
smoking_status       object
stroke                int64
dtype: object

### Define the X and y so that the X are the Features and y is the targets

In [71]:
df.head()

gender  age  hypertension  heart_disease ever_married      work_type  \
0    Male   67             0              1          Yes        Private   
1  Female   61             0              0          Yes  Self-employed   
2    Male   80             0              1          Yes        Private   
3  Female   49             0              0          Yes        Private   
4  Female   79             1              0          Yes  Self-employed   

  Residence_type  avg_glucose_level  bmi   smoking_status  stroke  
0          Urban                228   36  formerly smoked       1  
1          Rural                202   28     never smoked       1  
2          Rural                105   32     never smoked       1  
3          Urban                171   34           smokes       1  
4          Rural                174   24     never smoked       1

In [72]:
X = df.drop('stroke', axis=1)
X.head()

gender  age  hypertension  heart_disease ever_married      work_type  \
0    Male   67             0              1          Yes        Private   
1  Female   61             0              0          Yes  Self-employed   
2    Male   80             0              1          Yes        Private   
3  Female   49             0              0          Yes        Private   
4  Female   79             1              0          Yes  Self-employed   

  Residence_type  avg_glucose_level  bmi   smoking_status  
0          Urban                228   36  formerly smoked  
1          Rural                202   28     never smoked  
2          Rural                105   32     never smoked  
3          Urban                171   34           smokes  
4          Rural                174   24     never smoked

In [73]:
y = df['stroke']
y.head()

0    1
1    1
2    1
3    1
4    1
Name: stroke, dtype: int64

### Split the data into Testing, Training

In [90]:
from sklearn.pipeline import Pipeline
from imblearn.pipeline import make_pipeline
#from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from imblearn.combine import SMOTETomek
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import SMOTENC
from sklearn.linear_model import LogisticRegression
from sklearn.compose import make_column_transformer
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.compose import make_column_transformer
from imblearn.over_sampling import SMOTENC
from sklearn.pipeline import FeatureUnion
from sklearn import preprocessing





In [75]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)


In [76]:
X_train.head()

gender  age  hypertension  heart_disease ever_married work_type  \
845   Female   48             0              0          Yes   Private   
3744  Female   29             0              0           No   Private   
4183  Female   35             0              0          Yes   Private   
3409    Male   38             0              0          Yes   Private   
284     Male   14             0              0           No  Govt_job   

     Residence_type  avg_glucose_level  bmi smoking_status  
845           Urban                 69   33   never smoked  
3744          Urban                 84   21   never smoked  
4183          Rural                119   22   never smoked  
3409          Rural                108   32   never smoked  
284           Urban                 82   31        Unknown

### Logistic Regression Pipeline

example taken from the imbalanced learn documentation:

https://imbalanced-learn.org/stable/auto_examples/pipeline/plot_pipeline_classification.html


In [77]:
X_train.columns

Index(['gender', 'age', 'hypertension', 'heart_disease', 'ever_married',
       'work_type', 'Residence_type', 'avg_glucose_level', 'bmi',
       'smoking_status'],
      dtype='object')

In [114]:
smote = SMOTENC(X_train)
LR = LogisticRegression()
ss = StandardScaler()
ohe = OneHotEncoder()
column_trans = make_column_transformer(
    (OneHotEncoder(),['gender', 'hypertension', 'heart_disease', 
                      'ever_married', 'work_type', 'Residence_type', 
                      'smoking_status']), remainder='passthrough')


In [108]:
X_train.head()

gender  age  hypertension  heart_disease ever_married work_type  \
845   Female   48             0              0          Yes   Private   
3744  Female   29             0              0           No   Private   
4183  Female   35             0              0          Yes   Private   
3409    Male   38             0              0          Yes   Private   
284     Male   14             0              0           No  Govt_job   

     Residence_type  avg_glucose_level  bmi smoking_status  
845           Urban                 69   33   never smoked  
3744          Urban                 84   21   never smoked  
4183          Rural                119   22   never smoked  
3409          Rural                108   32   never smoked  
284           Urban                 82   31        Unknown

In [111]:
# Make Pipeline Command

pipe = make_pipeline(column_trans, smote, ss, LR)

In [112]:
pipe.fit(X_train, y_train)

ValueError: Some of the categorical indices are out of range. Indices should be between 0 and 22

### Classification report

In [ ]:
y_pred_lr = pipe.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred_lr))

![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)

### Junaid's Comments:

So to be explicit, the process would be:

1. Split data into training, validation and test
2. preprocess the data (all splits)
3. If there is a class imbalance, perform up sampling or downsampling (usually via the SMOTE method) on the training data ONLY
4. fit model on training data
5. Perform hyper parameter tuning using the validation set as checks to see if model performance is improving - be careful to avoid overfitting
6. Once you have a final model you’re happy with, compute its adequacy metrics on the test data set

![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)